In [1]:
import weaviate
from weaviate.config import AdditionalConfig
from weaviate.classes.config import Configure

import json
import pandas as pd
import numpy as np
import os
import dotenv


from langchain_weaviate.vectorstores import WeaviateVectorStore
from langchain_openai import AzureOpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_openai.chat_models import AzureChatOpenAI  

dotenv.load_dotenv("~/.env")

AZURE_OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
AZURE_OPENAI_ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')
AZURE_OPENAI_API_VERSION = os.getenv('AZURE_OPENAI_API_VERSION')
AZURE_OPENAI_RESOURCE_NAME = os.getenv('AZURE_OPENAI_RESOURCE_NAME')

client = weaviate.connect_to_local(
    port=8083,
    grpc_port=50051,
    headers = {
        "X-Azure-Api-Key": AZURE_OPENAI_API_KEY,
    },
)

print(client.is_ready())


True


In [ ]:
df = pd.read_parquet("compared.parquet")
df

In [ ]:
print(df)

In [ ]:
from weaviate.classes.query import Filter


In [ ]:
COLLECTION_NAME = "recipes"
collection = client.collections.get(name=COLLECTION_NAME)

for item in collection.iterator():
    print(item.uuid, item.properties)

In [ ]:
COLLECTION_NAME = "recipes"
collection = client.collections.get(name=COLLECTION_NAME)

for document_uuid in df.document_sha.unique():

    collection.data.delete_many(
       where=Filter.by_id().contains_any(['108bda91-73bf-55ff-9c80-510e547ea471']) 
    )

In [2]:
from weaviate.util import generate_uuid5
generate_uuid5("hola")

'f2f96c1c-64fa-5520-a058-85e2e0d5e0c1'

In [5]:
len(generate_uuid5("cssfsdgsdf").replace("-", ""))

32